In [1]:
import os

In [ ]:

SCRIPT_DIR = os.path.dirname(os.path.abspath('/nlsasfs/home/gpucbh/vyakti11/iitrpr11/tasks'))

DATA_DIR = os.path.join(SCRIPT_DIR, 'data/Train')

DATA_DIR = os.path.normpath(DATA_DIR)

In [ ]:
import os
import sys
import cv2
import numpy as np
import pandas as pd
from scipy.signal import detrend

def POS_WANG(signal, fps):
    """
    Plain-Orthogonal-to-Skin (POS) Algorithm.
   
    Args:
        signal (np.array): Input RGB signal with shape (Frames, 3)
        fps (float): Frames per second
       
    Returns:
        np.array: The extracted rPPG waveform
    """

    C = signal.T  
   
    mean_C = np.mean(C, axis=1, keepdims=True)
    C = C / mean_C - 1

    X = C[1, :] - C[2, :]  
    Y = C[1, :] + C[2, :] - 2 * C[0, :] 

    X_std = np.std(X)
    Y_std = np.std(Y)
   
    if Y_std == 0:
        h = X
    else:
        alpha = X_std / Y_std
        h = X + alpha * Y

    h = detrend(h)
    return h

DATA_DIR = os.path.normpath(DATA_DIR)

META_FILE = os.path.join(DATA_DIR, "labels_train.xlsx")

print(f" Script Location: {SCRIPT_DIR}")
print(f" Data Location:   {DATA_DIR}")

if not os.path.exists(DATA_DIR):
    print(f" Error: The directory '{DATA_DIR}' does not exist.")
    print("Please ensure your folder structure is:")
    print("  /ParentFolder")
    print("     |-- /task (contains this script)")
    print("     |-- /data (contains videos)")
    sys.exit() # Stop execution if data is missing

# ---- 3. FACE DETECTION ----
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

def extract_face(frame):
    """Detects the largest face in the frame."""
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    if len(faces) == 0:
        return None

    x, y, w, h = max(faces, key=lambda b: b[2] * b[3])
    return frame[y:y+h, x:x+w]

def extract_rgb_signal(video_path):
    """Extracts the mean RGB signal from the face ROI."""
    cap = cv2.VideoCapture(video_path)
    rgb_signals = []

    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}")
        return np.array([])

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        face = extract_face(frame)
       
        if face is None:
            continue

        try:
            face = cv2.resize(face, (128, 128))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            mean_rgb = np.mean(face.reshape(-1, 3), axis=0)
            rgb_signals.append(mean_rgb)
        except Exception:
            continue

    cap.release()
    return np.array(rgb_signals)

def main():
    if not os.path.exists(META_FILE):
        print("Notice: Metadata file not found. Scanning 'data' folder for videos...")
        video_files = [f for f in os.listdir(DATA_DIR) if f.endswith(('.mp4', '.avi', '.mov'))]
       
        if not video_files:
            print(f"Error: No videos found in {DATA_DIR}.")
            return
           
        df = pd.DataFrame({'video': video_files, 'label': [0] * len(video_files)})
    else:
        df = pd.read_excel(META_FILE)

    results = {}

    for _, row in df.iterrows():
        video_name = row["video"]
        label = row["label"]
        video_path = os.path.join(DATA_DIR, video_name)

        print(f"Processing: {video_name}...")

        if not os.path.exists(video_path):
            print(f"Warning: File not found ({video_name}). Skipping.")
            continue

        cap = cv2.VideoCapture(video_path)
        fps = cap.get(cv2.CAP_PROP_FPS)
        cap.release()

        rgb_signal = extract_rgb_signal(video_path)

        if len(rgb_signal) < fps:
            print(f"Skipping {video_name}: Signal too short or face not detected.")
            continue

        try:
            rppg_waveform = POS_WANG(rgb_signal, fps)
           
            results[video_name] = {
                "label": label,
                "rppg": rppg_waveform
            }
            print(f"Success: {video_name} | Waveform Length: {len(rppg_waveform)}")

        except Exception as e:
            print(f"Error processing {video_name}: {e}")

    output_path = os.path.join(DATA_DIR, "rppg_waveforms_pos.npy")
    if results:
        np.save(output_path, results)
        print(f"\nAll rPPG signals have been saved to: {output_path}")
    else:
        print("\nNo data was saved.")

if __name__ == "__main__":
    main()

📂 Script Location: /nlsasfs/home/gpucbh/vyakti11/iitrpr11
📂 Data Location:   /nlsasfs/home/gpucbh/vyakti11/iitrpr11/data/Train
Processing: subject_10_Vid_6.avi...
Processing: subject_11_Vid_6.MP4...
Processing: subject_12_Vid_6.MP4...
Processing: subject_15_Vid_6.avi...
Processing: subject_16_Vid_6.MP4...
Processing: subject_18_Vid_6.webm...
Processing: subject_1_Vid_5.avi...
Processing: subject_20_Vid_2.avi...
Success: subject_20_Vid_2.avi | Waveform Length: 2719
Processing: subject_20_Vid_5_2.avi...
Processing: subject_20_Vid_6.avi...
Success: subject_20_Vid_6.avi | Waveform Length: 2407
Processing: subject_20_Vid_7.avi...
Processing: subject_21_Vid_5.avi...
Processing: subject_23_Vid_5.avi...
Success: subject_23_Vid_5.avi | Waveform Length: 5081
Processing: subject_3_Vid_2.avi...
Processing: subject_3_Vid_5.avi...
Processing: subject_3_Vid_7.avi...
Processing: subject_4_Vid_6.MP4...
Processing: subject_6_Vid_6.MP4...
Processing: subject_7_Vid_2.avi...
Processing: subject_7_Vid_4.avi

[matroska,webm @ 0x5639c193fc40] File ended prematurely at pos. 12082631 (0xb85dc7)
[opus @ 0x5639c18f1500] Error parsing Opus packet header.


Success: subject_52_Vid_7.webm | Waveform Length: 9172
Processing: subject_53_Vid_3.avi...
Success: subject_53_Vid_3.avi | Waveform Length: 820
Processing: subject_56_Vid_3.avi...
Success: subject_56_Vid_3.avi | Waveform Length: 513
Processing: subject_62_Vid_3.avi...
Success: subject_62_Vid_3.avi | Waveform Length: 3094
Processing: subject_62_Vid_4.avi...
Success: subject_62_Vid_4.avi | Waveform Length: 3270
Processing: subject_62_Vid_5.avi...
Success: subject_62_Vid_5.avi | Waveform Length: 2875
Processing: subject_62_Vid_7.avi...
Processing: subject_64_Vid_7.wmv...
Success: subject_64_Vid_7.wmv | Waveform Length: 4846
Processing: subject_65_Vid_6.avi...
Processing: subject_66_Vid_7.wmv...
Success: subject_66_Vid_7.wmv | Waveform Length: 567
Processing: subject_67_Vid_1.avi...
Success: subject_67_Vid_1.avi | Waveform Length: 3655
Processing: subject_69_Vid_6.wmv...
Success: subject_69_Vid_6.wmv | Waveform Length: 3679
Processing: subject_70_Vid_1.avi...
Processing: subject_70_Vid_2.a